### 9.4 Richardson Extrapolation

In [ ]:
import numpy as np
from scripts.interpolation import neville, difference_1_forward, difference_1_central, difference_2_central

#### Example 9.29 (Extrapolation of the forward difference quotient)

Let $f(x)=\tanh(x)$. We want to determine the derivative of $f$ at $x_0=0.5$. The exact value is $f'(1/2) \approx 0.786448$. For the numerical approximation, we use the forward difference and use step sizes $h=2^{-1}, 2^{-2}, 2^{-3}, 2^{-4}$.

In [ ]:
data = [[0.5**i, difference_1_forward(np.tanh, 0.5, 0.5**i)] for i in range(1, 5)]
data = np.array(data)
print(data)

Now we can interpolate the result with a polynomial $p(h)$ with nodes $h_i=2^{-i}$ and values $a_i=a(h_i)$, and evaluate this at $h=0$. This can be done efficiently with the Neville scheme.

In [ ]:
vals = neville(data, 0)
print(vals)

This results in the following errors

In [ ]:
print(np.triu(vals - 1 / np.cosh(0.5)**2))

Extrapolation does indeed improve the accuracy of the approximation.

#### Example 9.31 (Extrapolation of the central difference quotient)

We repeat the same example with the central difference quotient.

In [ ]:
data = [[0.5**i, difference_1_central(np.tanh, 0.5, 0.5**i)] for i in range(1, 5)]
data = np.array(data)
print(data)

In [ ]:
vals = neville(data, 0)
print(vals)

This results in the following errors

In [ ]:
print(np.triu(vals - 1 / np.cosh(0.5)**2))

#### Example 9.32 (Extrapolation of the central difference quotient with quadratic polynomials)

We begin by implementing the modified Neville scheme to realize the Richardson extrapolation

In [ ]:
def extrapolate(data, q):
    n = data.shape[0]
    h = data[:, 0]
    a = np.zeros((n, n))
    a[:, 0] = data[:, 1]
    
    for i in range(1, n):
        for k in range(1, i + 1):
            a[i, k] = a[i, k - 1] + (a[i, k - 1] - a[i - 1, k - 1]) / ((h[i - k] / h[i])**q - 1)
    return a

We use the Richardson extrapolation on the central difference quotient with order $q=2$.

In [ ]:
vals = extrapolate(data, 2)
print(vals)

This results in the following errors

In [ ]:
print(np.tril(vals - 1 / np.cosh(0.5)**2))

#### Example 9.35 (Extrapolation with different error orders)

We consider
$$f(x) = -e^{1-\cos(\pi x)}$$
and approximate $f''(1)$ with the central difference quotient.

In [ ]:
data = [[0.5**i, difference_2_central(lambda x: - np.exp(1 - np.cos(np.pi * x)), 1, 0.5**i)]
        for i in range(8)]
f2_ex = np.pi**2 * np.exp(2)
data = np.array(data)
print(data)

Using a linear extrapolation on these values, we get

In [ ]:
vals = extrapolate(data, 1)[:,:3]
print(vals)

We see that the first step of the extrapolation does not improve the order of convergence:

In [ ]:
err = np.abs(np.tril(vals - f2_ex))
out = np.zeros((err.shape[0], 6))
out[:, 0] = err[:, 0]
out[:, 2] = err[:, 1]
out[:, 4] = err[:, 2]
for j in range(3):
    for i in range(1 + j, err.shape[0]):
        out[i, 2 * j + 1] = (np.log(err[i - 1][j]) - np.log(err[i][j])) / np.log(2)
print(out.astype(np.half))

By using the correct order (2) in the extrapolation, we see a clear improvement in the values

In [ ]:
vals = extrapolate(data, 2)[:,:3]
print(vals)

We see that after one extrapolation step, we get convergence order four and with two steps even convergence order six.

In [ ]:
err = np.abs(np.tril(vals - f2_ex))
out = np.zeros((err.shape[0], 6))
out[:, 0] = err[:, 0]
out[:, 2] = err[:, 1]
out[:, 4] = err[:, 2]
for j in range(3):
    for i in range(1 + j, err.shape[0]):
        out[i, 2 * j + 1] = (np.log(err[i - 1][j]) - np.log(err[i][j])) / np.log(2)
print(out.astype(np.half))